In [ ]:
import json
import os

import chromadb
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from chromadb.utils import embedding_functions
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer

In [ ]:
# https://docs.trychroma.com/guides
CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
# EMBED_MODEL = "all-mpnet-base-v2"
COLLECTION_NAME = "arxiv_papers"
BATCH_SIZE = 5000

CHROMA_DATA_PATH = os.path.join(CHROMA_DATA_PATH, EMBED_MODEL)

In [ ]:
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

parquet_path = '../data/arxiv_metadata_sample.parquet.gzip'
arxiv_df = pd.read_parquet(parquet_path)

print(arxiv_df.shape)

In [ ]:
def text_processing(sample):
    title = sample['title']
    abstract = sample['abstract']

    # remove special characters
    title = title.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    abstract = abstract.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    # remove multiple spaces
    title = ' '.join(title.split())
    abstract = ' '.join(abstract.split())

    return f"Title: {title} - Abstract: {abstract}"

In [ ]:
arxiv_df['text'] = arxiv_df.apply(text_processing, axis=1)
arxiv_df.head(3)

In [ ]:
def create_metadatas(arxiv_df):
    metadatas = []
    for _, row in arxiv_df.iterrows():
        metadatas.append({
            "update_date": row['update_date'],
            "title_words": row['title_words'],
            "abstract_words": row['abstract_words']
        })
        categories = {f"category_{i}": category for i, category in enumerate(row['mapped_categories'])}
        metadatas[-1].update(categories)

    return metadatas

In [ ]:
def create_collection(client, collection_name, embedding_function):
    collection = client.create_collection(
        name=collection_name,
        embedding_function=embedding_function,
        metadata={"hnsw:space": "cosine"},
        get_or_create=True,
    )

    return collection

def delete_collection_data(client, collection, collection_name):
    print(f"Deleting data from collection {collection_name} with {collection.count()} documents")
    client.delete_collection(collection_name)

def get_random_samples_from_collection(collection, n_samples):
    collection_ids = collection.get()["ids"]
    random_ids = np.random.choice(collection_ids, n_samples, replace=False).tolist()
    documents = collection.get(ids=random_ids)
    return documents

def upsert_data(collection, arxiv_df, metadatas, batch_size):
    for i in tqdm(range(0, len(arxiv_df), batch_size)):
        collection.upsert(
            documents=arxiv_df['text'].iloc[i:i + batch_size].tolist(),
            ids=arxiv_df['id'].iloc[i:i + batch_size].tolist(),
            metadatas=metadatas[i:i + batch_size],
        )

In [ ]:
# delete the collection if it exists
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL,
    device="cuda",
)

collection = create_collection(client, COLLECTION_NAME, embedding_func)

# delete if you want to start fresh but then you need to create the collection again
# delete_collection_data(client, collection, COLLECTION_NAME)
# collection = create_collection(client, COLLECTION_NAME, embedding_func)

# create metadatas
metadatas = create_metadatas(arxiv_df)

# upsert data (insert or update if exists)
# upsert_data(collection, arxiv_df, metadatas, BATCH_SIZE)

In [ ]:
# Define query
words_per_line = 10
# define papers to show
top_n_papers = 3
query = "The meaning of life in philosophy and the analysis of deep learning language"
print("Query:\n", query, "\n")
query_results = collection.query(query_texts=[query], n_results=top_n_papers)
for _id, _doc, _dist, _meta in zip(query_results["ids"][0], query_results["documents"][0], query_results["distances"][0], query_results["metadatas"][0]):
    print(f"#####   ID: {_id}   #####")
    print(f"Distance: {_dist}")
    print(f"Metadata: {_meta}")
    _doc_lines = _doc.split()
    for i in range(0, len(_doc_lines), words_per_line):
        print(" ".join(_doc_lines[i:i + words_per_line]))
    print("\n")

In [ ]:
# response = collection.get(include=["metadatas", "documents", "embeddings"])
df = pd.DataFrame({
    "id": response["ids"],
    "document": response["documents"],
    "embedding": response["embeddings"],
    "categories": [m["category_0"] for m in response["metadatas"]],
})
all_categories = df["categories"].explode().unique()
cat_mapping = {cat: i for i, cat in enumerate(all_categories)}
df["cat_id"] = df["categories"].apply(lambda x: cat_mapping[x])
df.loc[:, all_categories] = df["categories"].apply(lambda x: [1 if cat in x else 0 for cat in all_categories]).tolist()
df.head(3)

In [ ]:
# from renumics import spotlight

# spotlight.show(df)

# Simple Classification Head

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


class EmbeddingDataset(Dataset):
    def __init__(self, df, cat_ids):
        self.ids = df["id"].values
        self.embeddings = torch.tensor(df["embedding"].tolist(), dtype=torch.float32)
        self.categories = torch.tensor(df["cat_id"].values, dtype=torch.long)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return self.ids[idx], self.embeddings[idx], self.categories[idx]

class EmbeddingModel(torch.nn.Module):
    def __init__(self, n_emb_size, n_categories):
        super(EmbeddingModel, self).__init__()
        self.fc1 = torch.nn.Linear(n_emb_size, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, n_categories)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# train val split
emb_dataset = EmbeddingDataset(df, cat_mapping)
train_size = int(0.8 * len(emb_dataset))
val_size = len(emb_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(emb_dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}")
print(f"Val size: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmbeddingModel(len(df["embedding"].values[0]), len(all_categories)).to(device)

In [ ]:
def train(model, train_loader, val_loader, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for _, emb, cat in train_loader:
            emb, cat = emb.to(device), cat.to(device)
            optimizer.zero_grad()
            output = model(emb)
            loss = criterion(output, cat)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for _, emb, cat in val_loader:
                emb, cat = emb.to(device), cat.to(device)
                output = model(emb)
                _, predicted = torch.max(output.data, 1)
                total += cat.size(0)
                correct += (predicted == cat).sum().item()

            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy: {100 * correct / total}")

train(model, train_loader, val_loader, epochs=10)

# cluster

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

classes = arxiv_df['mapped_categories'].explode().unique()
classes = {c: i for i, c in enumerate(classes)}
num_classes = len(classes)
print(f"Number of classes: {num_classes}")
arxiv_df['class'] = arxiv_df['mapped_categories'].apply(lambda x: classes[x[0]])

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=num_classes, n_init=4, random_state=0)
estimator = make_pipeline(StandardScaler(), kmeans).fit(df["embedding"].tolist())
# estimator = make_pipeline(kmeans).fit(df["embedding"].tolist())

df["cluster"] = estimator.predict(df["embedding"].tolist())
merged_df = pd.merge(df[['id', 'cluster']], arxiv_df, on="id")

accuracy = metrics.accuracy_score(merged_df['class'], merged_df['cluster'])
print(f"Accuracy: {accuracy}")